In [1]:
!pip install numpy
!pip install pandas
!pip install mealpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 62.3 MB/s eta 0:00:00


In [16]:
import pandas as pd
import numpy as np
import time
from mealpy.utils.space import IntegerVar
from mealpy.swarm_based import MPA

# Function to generate datasets
def generate_datasets(num_items, num_slots, seed=42, size="Small"):
    np.random.seed(seed)

    item_data = {
        "ItemID": [f"Item_{i+1}" for i in range(num_items)],
        "DemandFrequency": np.random.choice(["High", "Medium", "Low"], num_items, p=[0.4, 0.4, 0.2]),
        "Size": np.random.choice(["Small", "Medium", "Large"], num_items, p=[0.5, 0.3, 0.2]),
        "FrequentPair": [f"Item_{np.random.randint(1, num_items+1)}" if np.random.rand() > 0.7 else None for _ in range(num_items)],
    }
    items_df = pd.DataFrame(item_data)
    items_file = f'{size}_items_dataset.csv'
    items_df.to_csv(items_file, index=False)
    print(f"Datasets saved as '{items_file}'.")

    slot_data = {
        "SlotID": [f"Slot_{i+1}" for i in range(num_slots)],
        "SlotType": np.random.choice(
            ["Divisible (Small)", "Divisible (Medium)", "Adjustable (Large)", "Standard"],
            num_slots,
            p=[0.3, 0.3, 0.2, 0.2],
        ),
        "Proximity": np.random.choice(["Close", "Medium", "Far"], num_slots, p=[0.4, 0.4, 0.2]),
    }
    slots_df = pd.DataFrame(slot_data)
    slots_file = f'{size}_slots_dataset.csv'
    slots_df.to_csv(slots_file, index=False)
    print(f"Datasets saved as '{slots_file}'.")

    return items_df, slots_df

def fitness_function(solution, items_df, slots_df):
    total_distance = 0
    penalty = 0
    used_slots = set()

    for i, slot_index in enumerate(solution):
        item = items_df.iloc[i]
        if slot_index >= len(slots_df):
            penalty += 10
            continue

        slot = slots_df.iloc[int(slot_index)]

        # Add penalty if slot already used
        if slot["SlotID"] in used_slots:
            penalty += 5
        else:
            used_slots.add(slot["SlotID"])

        # Check size compatibility
        if item["Size"] == "Small" and "Small" not in slot["SlotType"] and "Standard" not in slot["SlotType"]:
            penalty += 3
        elif item["Size"] == "Medium" and "Medium" not in slot["SlotType"] and "Standard" not in slot["SlotType"]:
            penalty += 3
        elif item["Size"] == "Large" and "Large" not in slot["SlotType"] and "Standard" not in slot["SlotType"]:
            penalty += 3

        # Add distance based on proximity
        if item["DemandFrequency"] == "High":
            total_distance += 1 if slot["Proximity"] == "Close" else 5
        elif item["DemandFrequency"] == "Medium":
            total_distance += 3 if slot["Proximity"] == "Medium" else 5
        elif item["DemandFrequency"] == "Low":
            total_distance += 5 if slot["Proximity"] == "Far" else 10

    return total_distance + penalty

def optimize_allocation(items_df, slots_df, num_items, num_slots):
    bounds = [IntegerVar(0, num_slots - 1) for _ in range(num_items)]
    problem = {
        "obj_func": lambda solution: fitness_function(solution, items_df, slots_df),
        "bounds": bounds,
        "minmax": "min",
        "verbose": True,
    }
    MPA_model = MPA.OriginalMPA(epoch=50, pop_size=50)
    best_agent = MPA_model.solve(problem)
    return best_agent.solution, best_agent.target

# Warehouse sizes
warehouses = {
    "Small": {"num_items": 50, "num_slots": 35},
    "Medium": {"num_items": 100, "num_slots": 85},
    "Large": {"num_items": 150, "num_slots": 130},
}

# Number of runs to calculate average fitness
num_runs = 50

# Run optimization for each warehouse size
results = {}
for size, params in warehouses.items():
    print(f"\n--- Optimizing for {size} Warehouse ---")
    items_df, slots_df = generate_datasets(params["num_items"], params["num_slots"], size=size)

    fitness_values = []
    run_times = []

    for _ in range(num_runs):
        start_time = time.time()

        best_solution, best_fitness = optimize_allocation(items_df, slots_df, params["num_items"], params["num_slots"])

        run_time = time.time() - start_time
        run_times.append(run_time)

        # Append the raw fitness value
        fitness_values.append(best_fitness._fitness)
        print(best_fitness.__dict__)  # Print the attributes of the Target object

        # Create allocation result for each run
        allocation_result = pd.DataFrame({
            "ItemID": items_df["ItemID"],
            "AssignedSlot": [
                slots_df.iloc[int(slot_index)]["SlotID"] if slot_index < len(slots_df) else None
                for slot_index in best_solution
            ],
            "SlotType": [
                slots_df.iloc[int(slot_index)]["SlotType"] if slot_index < len(slots_df) else None
                for slot_index in best_solution
            ],
        })

    avg_fitness = np.mean(fitness_values)
    avg_runtime = np.mean(run_times)

    results[size] = {
        "avg_fitness": avg_fitness,
        "best_fitness": min(fitness_values),
        "allocation": allocation_result,
    }

    print(f"Best Fitness Score for {size} Warehouse: {min(fitness_values)}")
    print(f"Average Fitness Score for {size} Warehouse: {avg_fitness}")
    print(f"Average Runtime for {size} Warehouse: {avg_runtime} seconds")
    print(allocation_result.head())

# Save results to CSV files
from google.colab import files

for size, data in results.items():
    file_path = f"{size}_warehouse_allocation.csv"
    data["allocation"].to_csv(file_path, index=False)
    files.download(file_path)
    print(f"Downloaded {size} warehouse allocation as {file_path}")


INFO:mealpy.swarm_based.MPA.OriginalMPA:Solving single objective optimization problem.



--- Optimizing for Small Warehouse ---
Datasets saved as 'Small_items_dataset.csv'.
Datasets saved as 'Small_slots_dataset.csv'.


INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 1, Current best: 383.0, Global best: 383.0, Runtime: 0.30956 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 2, Current best: 383.0, Global best: 383.0, Runtime: 0.28097 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 3, Current best: 369.0, Global best: 369.0, Runtime: 0.29139 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 4, Current best: 356.0, Global best: 356.0, Runtime: 0.23187 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 5, Current best: 356.0, Global best: 356.0, Runtime: 0.18148 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 6, Current best: 356.0, Global best: 356.0, Runtime: 0.16913 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 7, Current best: 356.0, Global best: 356.0, Runtime: 0.17807 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 8, Current best: 356.

{'_objectives': array([292]), '_weights': array([1.]), '_fitness': 292.0}


INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 1, Current best: 381.0, Global best: 381.0, Runtime: 0.18383 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 2, Current best: 381.0, Global best: 381.0, Runtime: 0.16626 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 3, Current best: 381.0, Global best: 381.0, Runtime: 0.16965 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 4, Current best: 368.0, Global best: 368.0, Runtime: 0.16652 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 5, Current best: 364.0, Global best: 364.0, Runtime: 0.17221 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 6, Current best: 360.0, Global best: 360.0, Runtime: 0.18251 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 7, Current best: 360.0, Global best: 360.0, Runtime: 0.17960 seconds
INFO:mealpy.swarm_based.MPA.OriginalMPA:>>>Problem: P, Epoch: 8, Current best: 354.

...

<IPython.core.display.Javascript object>

Downloaded Medium warehouse allocation as Medium_warehouse_allocation.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded Large warehouse allocation as Large_warehouse_allocation.csv
